# Analiza mniej znaczących kolumn

###### Importy

In [1]:
from collections import defaultdict

In [2]:
import pandas as pd
import numpy as np

bestiary1 = pd.read_json("pathfinder_2e_data/pathfinder-bestiary.db", lines=True)
bestiary2 = pd.read_json("pathfinder_2e_data/pathfinder-bestiary-2.db", lines=True)
bestiary3 = pd.read_json("pathfinder_2e_data/pathfinder-bestiary-3.db", lines=True)
print(bestiary1.iloc[0])

_id                                                024PqcF8yMRBrPuq
img                        systems/pf2e/icons/default-icons/npc.svg
items             [{'_id': 'B7rzf0nBJmzg8x0y', 'img': 'systems/p...
name                                             Adult White Dragon
system            {'abilities': {'cha': {'mod': 1}, 'con': {'mod...
type                                                            npc
flags             {'core': {'sourceId': 'Compendium.pf2e.pathfin...
prototypeToken                                                  NaN
Name: 0, dtype: object


In [3]:
bestiary = pd.concat([bestiary1, bestiary2, bestiary3], join="outer", axis=0).fillna(
    np.nan
)

In [4]:
bestiary.shape

(1206, 8)

###### Funkcje pomocnicze

In [5]:
def unpack_column(df: pd.DataFrame, column_name: str):
    new_df = df[column_name].apply(pd.Series)
    print("First row:")
    print(new_df.iloc[0])
    print(f"Len = {len(new_df.iloc[0])}")
    return new_df

In [6]:
def print_unique_values(df: pd.DataFrame, col: str):
    print(f"Number of Nones: {df[col].isnull().sum()}")
    print(f"Number of unique values: {df[col].nunique()}")
    print("Unique values with number of occurences:")
    print(df[col].value_counts())

## Analiza

Id jest unikatową wartością dla każdego potwora, nie ma wpływu na level. Obrazek również nie wpływa na level.

#### ```_id```, ```image``` można usunąć

### Analiza ```name```

In [7]:
bestiary["name"].isnull().sum()

0

###### Wszystkie nazwy potworów

In [8]:
all_names = pd.DataFrame(bestiary.loc[:, "name"])
print(all_names.to_string())

                                        name
0                         Adult White Dragon
1                               Brontosaurus
2                             Giant Anaconda
3                       Ancient Green Dragon
4                             Lantern Archon
5                                  Dark Naga
6                                  Gancanagh
7                                   Dullahan
8                      Deep Gnome Rockwarden
9                            Zombie Shambler
10                                Goblin Dog
11                          Lizardfolk Scout
12                                Homunculus
13                            Ghost Commoner
14                                       Roc
15                               Deinonychus
16                              Tarn Linnorm
17          Ancient Red Dragon (Spellcaster)
18                                  Zaramuun
19                        Young White Dragon
20                           Giant Moray Eel
21        

#### ```name``` można usunąć

### Analiza ```type```

In [9]:
bestiary["type"].isnull().values.any()

False

In [10]:
print_unique_values(bestiary, "type")

Number of Nones: 0
Number of unique values: 1
Unique values with number of occurences:
npc    1206
Name: type, dtype: int64


Wszystkie wartości są npc.

#### ```type``` można usunąć

### Analiza ```flags```

Nazwy są unikatowe, smoki mają wersję normalną i wersję spellcaster (optional rule)

In [11]:
flags = unpack_column(bestiary, "flags")

First row:
core    {'sourceId': 'Compendium.pf2e.pathfinder-besti...
Name: 0, dtype: object
Len = 1


In [12]:
core = unpack_column(flags, "core")

First row:
sourceId    Compendium.pf2e.pathfinder-bestiary.024PqcF8yM...
Name: 0, dtype: object
Len = 1


In [13]:
print(core)

                                              sourceId
0    Compendium.pf2e.pathfinder-bestiary.024PqcF8yM...
1    Compendium.pf2e.pathfinder-bestiary.05E3kkjoLZ...
2    Compendium.pf2e.pathfinder-bestiary.05wwpHHsBl...
3    Compendium.pf2e.pathfinder-bestiary.0FGz2eXm0S...
4    Compendium.pf2e.pathfinder-bestiary.0HjVFx8qIK...
..                                                 ...
360  Compendium.pf2e.pathfinder-bestiary-3.ytYKPtPo...
361  Compendium.pf2e.pathfinder-bestiary-3.zGco5Qmo...
362  Compendium.pf2e.pathfinder-bestiary-3.zGtfiKku...
363  Compendium.pf2e.pathfinder-bestiary-3.zMoJh88v...
364  Compendium.pf2e.pathfinder-bestiary-3.zdJgaVe6...

[1206 rows x 1 columns]


Nic ważnego

#### ```flags``` można usunąć

### Analiza ```prototypeToken```

In [14]:
bestiary["prototypeToken"].isnull().sum()

1121

In [15]:
prototypeToken = unpack_column(bestiary, "prototypeToken")

First row:
0       NaN
name    NaN
Name: 0, dtype: object
Len = 2


In [16]:
print_unique_values(prototypeToken, "name")

Number of Nones: 1121
Number of unique values: 67
Unique values with number of occurences:
Draconal                    5
Wizard Sponge               5
Chimera                     5
Living Graffiti             4
Living Rune                 4
                           ..
Young Copper Dragon         1
Ancient Copper Dragon       1
Adult Silver Dragon         1
Adult Crystal Dragon        1
Ancient Sovereign Dragon    1
Name: name, Length: 67, dtype: int64


In [27]:
non_null_prototypeToken = unpack_column(
    bestiary[bestiary["prototypeToken"].notnull()], "prototypeToken"
)

First row:
name    Ancient Red Dragon
Name: 17, dtype: object
Len = 1


In [28]:
print(non_null_prototypeToken)

                         name
17         Ancient Red Dragon
28        Ancient Blue Dragon
40         Young Green Dragon
70          Young Gold Dragon
71         Adult Black Dragon
..                        ...
323                  Draconal
325             Wizard Sponge
329                  Draconal
332  Ancient Sovereign Dragon
336               Living Rune

[85 rows x 1 columns]


##### Porównianie Smoka spellcastera ze swoim prototypem

In [19]:
dragon_spellcaster = bestiary.iloc[17]
print(dragon_spellcaster)

_id                                                2T2cN4j4EzfDAIEe
img                        systems/pf2e/icons/default-icons/npc.svg
items             [{'_id': '80jq5AttlXMgdD6Q', 'img': 'systems/p...
name                               Ancient Red Dragon (Spellcaster)
system            {'abilities': {'cha': {'mod': 7}, 'con': {'mod...
type                                                            npc
flags             {'core': {'sourceId': 'Compendium.pf2e.pathfin...
prototypeToken                       {'name': 'Ancient Red Dragon'}
Name: 17, dtype: object


In [20]:
dragon = bestiary.loc[bestiary["name"] == "Ancient Red Dragon"].squeeze()
print(dragon)

_id                                                9H11QHnoKz8hHKSM
img                        systems/pf2e/icons/default-icons/npc.svg
items             [{'_id': 'qaEGHfAlvrP6CGwS', 'img': 'systems/p...
name                                             Ancient Red Dragon
system            {'abilities': {'cha': {'mod': 7}, 'con': {'mod...
type                                                            npc
flags             {'core': {'sourceId': 'Compendium.pf2e.pathfin...
prototypeToken                                                  NaN
Name: 62, dtype: object


In [21]:
print(dragon_spellcaster["system"] == dragon["system"])

True


In [22]:
print(dragon_spellcaster["flags"] == dragon["flags"])

False


In [23]:
print(len(dragon_spellcaster["items"]))

58


In [24]:
print(len(dragon["items"]))

29


Obstawiam, że to jest ten sam smok, tylko spellcaster ma dodatkowo wpisane spelle. W każdym razie ta kolumna jest do wyrzucenia.

#### ```prototypeToken``` można usunąć